# Employee Data Visualization

In [7]:
#import dependencies
import pandas as pd
from config import postgres_url

#import sql engine
from sqlalchemy import create_engine
engine = create_engine(postgres_url)
connection = engine.connect()

In [9]:
#create employee dataframe from sql query
query = "SELECT * FROM employees"
employees_df = pd.read_sql_query(query,con=engine)
employees_df.head()

,emp_no,emp_title_id,birth_date,first_name,last_name,sex,hire_date
0,473302,s0001,1953-07-25,Hideyuki,Zallocco,M,1990-04-28
1,475053,e0002,1954-11-18,Byong,Delgrande,F,1991-09-07
2,57444,e0002,1958-01-30,Berry,Babb,F,1992-03-21
3,421786,s0001,1957-09-28,Xiong,Verhoeff,M,1987-11-26
4,282238,e0003,1952-10-28,Abdelkader,Baumann,F,1991-01-18


In [10]:
#create salary dataframe from sql query
query = "SELECT * FROM salaries"
salary_df = pd.read_sql_query(query,con=engine)
salary_df.head()

,emp_no,salary
0,10001,60117
1,10002,65828
2,10003,40006
3,10004,40054
4,10005,78228


In [11]:
#merging the two dataframes
emp_salary_df = employees_df.merge(salary_df, on='emp_no')
emp_salary_df.head()

,emp_no,emp_title_id,birth_date,first_name,last_name,sex,hire_date,salary
0,473302,s0001,1953-07-25,Hideyuki,Zallocco,M,1990-04-28,40000
1,475053,e0002,1954-11-18,Byong,Delgrande,F,1991-09-07,53422
2,57444,e0002,1958-01-30,Berry,Babb,F,1992-03-21,48973
3,421786,s0001,1957-09-28,Xiong,Verhoeff,M,1987-11-26,40000
4,282238,e0003,1952-10-28,Abdelkader,Baumann,F,1991-01-18,40000


Create a histogram to visualize the most common salary ranges for employees.